In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/car-sales-extended/car-sales-extended.csv
/kaggle/input/car-sales-missing-data/car-sales-extended-missing-data.csv


In [2]:
import pandas as pd 
import numpy as np 
%matplotlib inline 
import matplotlib.pyplot as plt 


In [3]:
car_sales = pd.read_csv("/kaggle/input/car-sales-extended/car-sales-extended.csv")


In [4]:
x = car_sales.drop("Price" , axis = 1)

y = car_sales["Price"]



In [5]:
# Turn the categories from the data into numbers 

from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 

# We are treating Doors as the catogrical 

categorical_features = ["Make" , "Colour" , "Doors"]


# hot encoding is a process used to turn categories into numbers 
    
one_hot = OneHotEncoder(sparse_output=False)



# telling the transformer to take the hot encoder and apply it to the "categorical 
# feature" and passthrough remaing columns 
transformer = ColumnTransformer([("one_hot" , 
                                 one_hot  , 
                                 categorical_features)] , 
                                 remainder = "passthrough")

transformed_x = transformer.fit_transform(x)
transformed_x

pd.DataFrame(transformed_x)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,35431.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,192714.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,84714.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,154365.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,181577.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,35820.0
996,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,155144.0
997,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,66604.0
998,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,215883.0


In [6]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

# lets refit the model 

from sklearn.model_selection import train_test_split 
x_train , x_test , y_train , y_test = train_test_split(transformed_x , y , test_size = 0.2)

model.fit(x_train , y_train)

RandomForestRegressor()

In [7]:
model.score(x_train , y_train)

0.8982626422226283

In [8]:
model.score(x_test , y_test)

0.19350908576706016

### 1. What if there were missing values 

1. Fill them with some values (also known as imputation).
2. Remove the data with missing samples altogether.

In [9]:
car_sales_missing = pd.read_csv("/kaggle/input/car-sales-missing-data/car-sales-extended-missing-data.csv")

car_sales_missing.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0


In [10]:
# need to fill missing data 
car_sales_missing.isna().sum()


Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

# option1 : filling missing data with panda 

In [11]:
# Filling the Make column 
# Replace missing values in "Make" with the string "missing"
car_sales_missing["Make"] = car_sales_missing["Make"].fillna("missing")

# Colour column 
# Replace missing values in "Colour" with the string "missing"
car_sales_missing["Colour"] = car_sales_missing["Colour"].fillna("missing")

# Odometer (KM)
# Replace missing values in "Odometer (KM)" with the mean of the column
car_sales_missing["Odometer (KM)"] = car_sales_missing["Odometer (KM)"].fillna(car_sales_missing["Odometer (KM)"].mean())

# Doors 
# Replace missing values in "Doors" with the constant value 4
car_sales_missing["Doors"] = car_sales_missing["Doors"].fillna(4)


In [12]:
# All the feature columns are done 
car_sales_missing.isna().sum()

# For the missing values of price its hard to predict so we will remove those rows

car_sales_missing = car_sales_missing.dropna(subset=["Price"])



In [13]:
car_sales_missing.isna().sum()

Make             0
Colour           0
Odometer (KM)    0
Doors            0
Price            0
dtype: int64

In [14]:
x = car_sales_missing.drop("Price" , axis = 1)
y = car_sales_missing["Price"]


In [15]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = ["Make" , "Colour" , "Doors"]


# by default sparse_output is false so it returns sparse matrix 
one_hot = OneHotEncoder(sparse_output=False)

transformer = ColumnTransformer([("one_hot",
                                one_hot,
                                categorical_features)] , 
                               remainder ="passthrough")


transformed_x = transformer.fit_transform(x)
transformed_x



array([[0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 3.54310e+04],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        1.00000e+00, 1.92714e+05],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 8.47140e+04],
       ...,
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 6.66040e+04],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 2.15883e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 2.48360e+05]])

In [16]:
from sklearn.model_selection import train_test_split 

x_train , x_test , y_train , y_test = train_test_split(transformed_x , y)

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

model.fit(x_train , y_train)

model.score(x_train , y_train)




0.8781503631468979

In [17]:
model.score(x_test , y_test)

0.14356907852523604

# Option 2 . Fill missing values with Scikit-learn 